<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/notebooks/retriever/BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [**Okapi BM25**](https://en.wikipedia.org/wiki/Okapi_BM25)
_BM_ 은 best matching의 약어로 확률적 정보 검색(IR: Information Retrieval) 및 키워드 검색에서 널리 쓰이는 문서 랭킹 함수이다.

## How
1. 쿼리에서 등장하는 단어가 문서에서 얼마나 자주 등장하는지 **(TF: Term Frequency)**
2. 그 단어가 전체 문서에서 얼마나 희귀한지 **(IDF: Inverse Document Frequency)**
3. 문서의 길이에 따라 가중치 부여 (길이가 긴 문서는 점수를 낮추는 방식)

</br>
$$
\text{score}(D, Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot
\frac{f(q_i, D) \cdot (k_1 + 1)}
{f(q_i, D) + k_1 \cdot \left(1 - b + b \cdot \frac{|D|}{\text{avgdl}}\right)}
$$
</br>

- $f(q_i, D)$ : 문서 $D$ 내의 특정 단어 $q_i$가 나타내는 횟수(TF)
- $|D|$ : 문서 $D$의 길이
- avgdl : 전체 문서의 평균 길이
- $k_1$과 $b$ : hyper parameters
  - $k_1$ : 키워드의 빈도에 따른 점수 영향력 억제, 무분별한 키워드 빈도 수 증가에 따른 중요도 향상 억제한다. (default 1.2 ~ 2.0)
  - $b$ : 문서 길이에 대한 중요도 제한, 짧은 문서에 특정 단어가 존재하면 중요도를 높게 평가한다. (default 0.75)

</br>
$$
\text{IDF}(q_i) = \ln \left( \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5} + 1 \right)
$$
</br>

- $N$ : 전체 문서 수
- $n(q_i)$ : 단어 $q_i$를 포함하는 문서의 수
- IDF(Inverse Document Frequency)는 특정 단어 $q_i$가 모든 문서에 존재한다면 중요하지 않다고 여기고 특정 문서에만 존재하는 단어라면 중요하다고 평가한다. **IDF가 낮으면 중요도가 낮고, IDF가 높으면 중요도가 높다.**


In [1]:
# BM25 from Scratch
import math
from collections import Counter

class BM25:
  def __init__(self, corpus, k1=1.2, b=0.75):
    self.corpus = corpus
    self.k1 = k1
    self.b = b
    self.N = len(corpus)
    self.avgdl = sum(len(doc) for doc in corpus) / self.N
    self.df = self._calculate_df()
    self.idf = self._calculate_idf()

  # Python에서 함수 앞에 _는 private이라는 관용 표현, 외부에서 호출하지 않는다는 의미
  ## Document Frequency
  def _calculate_df(self):
    df = {}
    for doc in self.corpus:
      for word in set(doc):
        # 딕셔너리 df에 word 키가 이미 있으면 +1, 없으면 0 + 1로 새로 추가
        df[word] = df.get(word, 0) + 1
    return df

  #Inverted Document Frequency
  def _calculate_idf(self):
    idf = {}
    for term, df in self.df.items():
        idf[term] = math.log((self.N - df + 0.5) / (df + 0.5) + 1)
    return idf

  def get_scores(self, query, doc_index):
    doc = self.corpus[doc_index]
    doc_len = len(doc)
    tf = Counter(doc)
    score = 0.0

    for q in query:
        if q not in tf:
            continue
        idf = self.idf.get(q, 0)
        freq = tf[q]
        denom = freq + self.k1 * (1 - self.b + self.b * (doc_len / self.avgdl))
        score += idf * ((freq * (self.k1 + 1)) / denom)
    return score

  def get_top_k(self, query, k=3):
    scores = [(i, self.get_scores(query, i)) for i in range(self.N)]
    return sorted(scores, key=lambda x: x[1], reverse=True)[:k]

In [2]:
corpus = [
    "the cat in the hat".split(),
    "the quick brown fox".split(),
    "the lazy dog and the fox".split()
]

query = "fox and dog"

bm25 = BM25(corpus)
top_docs = bm25.get_top_k(query.split())

for idx, score in top_docs:
    print(f"Doc {idx}: {corpus[idx]} → Score: {score:.4f}")

Doc 2: ['the', 'lazy', 'dog', 'and', 'the', 'fox'] → Score: 2.2478
Doc 1: ['the', 'quick', 'brown', 'fox'] → Score: 0.5119
Doc 0: ['the', 'cat', 'in', 'the', 'hat'] → Score: 0.0000


## **사용 사례**
- 검색 엔진의 기본 스코어링 알고리즘으로 채택
  - ElasticSearch, Whoosh 등
  - 키워드 기반의 문서 검색, 뉴스 검색, 제품 검색 등에 유용
- RAG에서 초기 후보 문서 추출
  - LLM 기반 시스템에서 빠르고 정확한 초기 필터링 역할

## **한계**
- ❌ 의미 기반 검색 불가
  - "강아지" vs "개", "자동차" vs "차량" → 단어가 다르면 매칭 불가
  - 동의어, 유사 개념, 문맥 이해 없음

- 🧠 벡터 의미 정보 없음
  - 단순히 단어 빈도 기반으로만 점수를 계산함
  - 문장 전체 의미를 파악하지 못함

RAG에서는 이러한 한계점을 극복하기 위해 **의미 기반 검색 기술을 혼합하여 Hybrid Search를 이용**한다.

# **BM25 라이브러리**
이전에는 BM25를 직접 구현해보았다면, 이제부터는 BM25를 미리 구현한 라이브러리나 탑재된 검색기에는 어떤 것이 있는지 확인해본다.

## **rank_bm25**
- BM25Okapi, BM25Plus, BM25L 지원
- 사용법이 매우 간단해서 빠르게 실험 가능

In [4]:
#!pip install -q rank_bm25

In [6]:
from rank_bm25 import BM25Okapi

corpus = [doc.split() for doc in ["the cat in the hat", "a quick brown fox", "lazy dog and fox"]]
bm25 = BM25Okapi(corpus)

query = "fox and dog".split()
scores = bm25.get_scores(query)

print(scores)

[0.         0.10823361 1.16651777]


## **ElasticSearch**
- 오픈소스 검색 서버
- 기본 점수 계산 모델이 BM25
- 역색인, 랭킹, 페이징 등 고급 기능 포함
- 실제 서비스에 검색 기능 붙일 때 대규모 문서나 웹 문서 검색에 적합

In [ ]:
#!pip install elasticsearch

In [9]:
# from elasticsearch import Elasticsearch

# es = Elasticsearch()
# query = {
#   "query": {
#     "match": {
#       "content": "fox and dog"
#     }
#   }
# }
# es.search(index="my_index", body=query)